# Scraping Data Twitter

**Proses install Library PyMysql**

In [ ]:
!pip install pymysql
!pip install mysql-connector-python-rf

# Load Data

Melakukan import library

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import glob

Membaca file CSV yang berisi scraping data Twitter pada periode sebelum terjadi COVID-19

In [ ]:
#tweet_data_before_covid_df = tweet_data_before_covid_df.dropna(how='any',axis=0)
tweet_data_before_covid_df = pd.DataFrame()

for files in glob.glob("data/noncovid/*.csv"):
    tweet_data_before_covid_files = pd.read_csv(files)
    tweet_data_before_covid_df = tweet_data_before_covid_df.append(tweet_data_before_covid_files)
    
#menghapus data rows yang redudant
tweet_data_before_covid_df = pd.DataFrame.drop_duplicates(tweet_data_before_covid_df)

Membaca file CSV yang berisi scraping data Twitter pada periode saat terjadi COVID-19

In [ ]:
#tweet_data_during_covid_df = tweet_data_during_covid_df.dropna(how='any',axis=0)
tweet_data_during_covid_df = pd.DataFrame()

for files in glob.glob("data/covid/*.csv"):
    tweet_data_during_covid_files = pd.read_csv(files)
    tweet_data_during_covid_df = tweet_data_during_covid_df.append(tweet_data_during_covid_files)
    
#menghapus data rows yang redudant
tweet_data_during_covid_df = pd.DataFrame.drop_duplicates(tweet_data_during_covid_df)

Menggabungkan data tweets sebelum dan saat COVID-19

In [ ]:
all_tweets_data_df = pd.concat([tweet_data_before_covid_df, tweet_data_during_covid_df], ignore_index=True)

Menghitung jumlah baris data keseluruhan

In [ ]:
print('Jumlah baris keseluruhan: ', len(all_tweets_data_df.index))

Jumlah baris keseluruhan:  1310835


In [ ]:
all_tweets_data_df.head(5)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1219771501609570305,1219771501609570305,2020-01-21 23:59:22 UTC,2020-01-21,23:59:22,0,56072466,harigelita,Dian Mochtar,NaN,...,"-6.216657128974757,106.83030289065285,10km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1219771450304786432,1219439859128922112,2020-01-21 23:59:10 UTC,2020-01-21,23:59:10,0,239827943,1dg4fux,Aryawipu,NaN,...,"-6.216657128974757,106.83030289065285,10km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Tospotato', 'name': 'Kamisam...",NaN,NaN,NaN,NaN
2,1219771440943091712,1219763519119220736,2020-01-21 23:59:08 UTC,2020-01-21,23:59:08,0,80840926,iwanneh,.:::.,NaN,...,"-6.216657128974757,106.83030289065285,10km",NaN,NaN,NaN,NaN,"[{'screen_name': 'NotesofMila', 'name': 'Mila🇮...",NaN,NaN,NaN,NaN
3,1219771347020083200,1219771347020083200,2020-01-21 23:58:46 UTC,2020-01-21,23:58:46,0,195299889,bys_king11,BODDAH,NaN,...,"-6.216657128974757,106.83030289065285,10km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1219771311368523776,1219771311368523776,2020-01-21 23:58:37 UTC,2020-01-21,23:58:37,0,406069088,anggi_gunawan25,Anggi Gunawan S.Tr.Sos,NaN,...,"-6.216657128974757,106.83030289065285,10km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


# Simpan Hasil Scraping Data ke MySQL

Load data hasil scraping Twitter yang sebelumnya, kemudian hilangkan NaN value

In [ ]:
all_tweets_data_df = all_tweets_data_df.replace(np.nan, '')

Lakukan pengecekan koneksi database

In [ ]:
import mysql.connector
from mysql.connector import errorcode

try:
  cnx = mysql.connector.connect(user='root', password='',
                                 host='localhost',
                                 database='twitter_db', charset="utf8mb4")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print('Connected')
    cnx.close()

Connected


Lakukan pembuatan Database untuk menyimpan datanya

In [ ]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', user='root', password='', charset="utf8mb4")
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS twitter_db")
        print("twitter_db database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

twitter_db database is created


Melakukan proses insert data hasil scraping dari Twitter ke database MySQL

In [ ]:
try:
    conn = msql.connect(host='localhost', database='twitter_db', user='root', password='', charset="utf8mb4")
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        print('Creating table....')
        cursor.execute("CREATE TABLE IF NOT EXISTS tweets ( \
                      `id` text DEFAULT NULL,\
                      `conversation_id` text DEFAULT NULL,\
                      `created_at` text DEFAULT NULL,\
                      `date` text DEFAULT NULL,\
                      `time` text DEFAULT NULL,\
                      `timezone` text DEFAULT NULL,\
                      `user_id` text DEFAULT NULL,\
                      `username` text DEFAULT NULL,\
                      `name` text DEFAULT NULL,\
                      `place` text DEFAULT NULL,\
                      `tweet` text DEFAULT NULL,\
                      `language` text DEFAULT NULL,\
                      `mentions` text DEFAULT NULL,\
                      `urls` text DEFAULT NULL,\
                      `photos` text DEFAULT NULL,\
                      `replies_count` text DEFAULT NULL,\
                      `retweets_count` text DEFAULT NULL,\
                      `likes_count` text DEFAULT NULL,\
                      `hashtags` text DEFAULT NULL,\
                      `cashtags` text DEFAULT NULL,\
                      `link` text DEFAULT NULL,\
                      `retweet` text DEFAULT NULL,\
                      `quote_url` text DEFAULT NULL,\
                      `video` text DEFAULT NULL,\
                      `thumbnail` text DEFAULT NULL,\
                      `near` text DEFAULT NULL,\
                      `geo` text DEFAULT NULL,\
                      `source` text DEFAULT NULL,\
                      `user_rt_id` text DEFAULT NULL,\
                      `user_rt` text DEFAULT NULL,\
                      `retweet_id` text DEFAULT NULL,\
                      `reply_to` text DEFAULT NULL,\
                      `retweet_date` text DEFAULT NULL,\
                      `translate` text DEFAULT NULL,\
                      `trans_src` text DEFAULT NULL,\
                      `trans_dest` text DEFAULT NULL\
                      ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_unicode_ci;")
        for i,row in tweet_data_before_covid_df.iterrows():
            sql = "INSERT INTO tweets VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL", e)

Melihat hasil data yang sudah dimasukkan

In [ ]:
try:
    conn = msql.connect(host='localhost', database='twitter_db', user='root', password='', charset="utf8mb4")
    if conn.is_connected():
        cursor = conn.cursor()
        # Execute query
        sql = "SELECT * FROM tweets limit 0,5"
        cursor.execute(sql)
        # Fetch all the records
        result = cursor.fetchall()
        for i in result:
            print(i)
except Error as e:
    print("Error while connecting to MySQL", e)        

('id', 'conversation_id', 'created_at', 'date', 'time', 'timezone', 'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video', 'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest')
('1247675354895904769', '1247675354895904769', '2020-04-07 23:59:19 UTC', '2020-04-07', '23:59:19', '+0000', '101723971', 'kimruby__', 'ev 🐨', '', 'ateez knp ya off stage gini bgt gabisa ☹️☹️☹️  https://t.co/CUFwelo3yW', 'in', '[]', '[]', '[]', '0', '0', '0', '[]', '[]', 'https://twitter.com/kimruby__/status/1247675354895904769', 'False', '', '1', 'https://pbs.twimg.com/ext_tw_video_thumb/1247553160178098177/pu/img/F-OGgiDDDOOzrAKY.jpg', '', '-6.216657128974757,106.83030289065285,10km', '', '', '', '', '[]', '', '', '', '')
('1247675351540482049', '1247662594598383616', '202